# Beiwe Sample Dataset Accelerometer EDA Pt. 2

Prototype helper functions to load and summarize an hour's worth of accelerometer data 


In [ ]:
import pandas as pd
import numpy as np

def summarize_hourly_file(file_path):
    """
    Analyze a single hourly accelerometer CSV file.
    
    Parameters:
    -----------
    file_path : str
        Path to the CSV file
    
    Returns:
    --------
    dict with keys:
        - n_rows
        - start_time
        - end_time
        - duration_min
        - sampling_min
        - duty_cycle
        - n_bursts
        - mean_magnitude
    """
    # TODO: Your code here
    pass  # Replace this